calculate the mean slope and elevation of primary wet forest in Haiti and the Dominican Republic 

In [1]:
import numpy as np
import os
from os.path import join
import pandas as pd
from osgeo import gdal, gdal_array, gdalconst

C:\Users\64937\AppData\Local\Temp\ipykernel_23164\2296229650.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
pwd = os.getcwd()
rootpath = os.path.abspath(os.path.join(pwd, '..'))

list_year = np.arange(1996, 2023)

img_country_mask = gdal_array.LoadFile(join(rootpath, 'data', 'hispaniola_polygon', 'countryid_hispaniola.tif'))

img_dem = gdal_array.LoadFile(join(rootpath, 'data', 'topography', 'dem_mosaic.tif'))
img_slope = gdal_array.LoadFile(join(rootpath, 'data', 'topography', 'slope_mosaic.tif'))

df_hispaniola_topography = pd.DataFrame(columns=['year', 'haiti_elevation_pwf', 'haiti_slope_pwf', 'dr_elevation_pwf', 'dr_slope_pwf'], 
                                        index=np.arange(0, len(list_year)))

for i_year in range(0, len(list_year)):
    year = list_year[i_year]

    df_hispaniola_topography.loc[i_year, 'year'] = year
    
    img_landcover = gdal_array.LoadFile(join(rootpath, 'data', 'hispaniola_lc', f'hispaniola_lc_{year}.tif'))
    
    # calculate the mean DEM and slope of primary wer forest for Haiti
    mask_landcover = (img_landcover == 2) & (img_country_mask == 1)
    mean_dem_haiti = np.nanmean(img_dem[mask_landcover])
    df_hispaniola_topography.loc[i_year, 'haiti_elevation_pwf'] = mean_dem_haiti

    mean_slope_haiti = np.nanmean(img_slope[mask_landcover])
    df_hispaniola_topography.loc[i_year, 'haiti_slope_pwf'] = mean_slope_haiti

    # calculate the mean DEM and slope of primary wer forest for Dominican Republic
    mask_landcover = (img_landcover == 2) & (img_country_mask == 2)
    mean_dem_dr = np.nanmean(img_dem[mask_landcover])
    df_hispaniola_topography.loc[i_year, 'dr_elevation_pwf'] = mean_dem_dr

    mean_slope_dr = np.nanmean(img_slope[mask_landcover])
    df_hispaniola_topography.loc[i_year, 'dr_slope_pwf'] = mean_slope_dr
    
    print(year, mean_dem_haiti, mean_slope_haiti, mean_dem_dr, mean_slope_dr)


1996 1597.6221 26.754847 1624.3041 21.413033
1997 1592.4376 26.868069 1618.3402 21.456446
1998 1588.3949 26.932268 1614.1033 21.479681
1999 1582.5527 26.957388 1609.3047 21.466652
2000 1568.6793 27.339785 1609.2573 21.481482
2001 1557.0089 27.596241 1606.44 21.463503
2002 1554.2931 27.678183 1605.6958 21.469503
2003 1553.5503 27.695702 1605.5806 21.480375
2004 1551.0409 27.763008 1605.338 21.485386
2005 1544.635 27.950556 1593.4507 21.442532
2006 1541.1876 28.030533 1583.0955 21.485048
2007 1536.3391 28.185392 1579.9475 21.539553
2008 1531.0721 28.364046 1579.114 21.556921
2009 1528.3241 28.44736 1579.3673 21.562342
2010 1526.7085 28.482862 1579.4371 21.56436
2011 1525.8435 28.511387 1579.6072 21.559895
2012 1521.0583 28.61349 1580.2488 21.564554
2013 1507.1781 29.054752 1580.8148 21.725647
2014 1506.2665 29.08857 1576.1101 21.783026
2015 1503.5137 29.157518 1573.5471 21.815952
2016 1496.1141 29.253853 1573.4967 21.830706
2017 1476.5585 29.269232 1573.4763 21.839706
2018 1476.3748 29.3

In [5]:
df_hispaniola_topography.to_excel(join(rootpath, 'results', 'hispaniola_pwf_topography.xlsx'))